In [1]:
from urllib.request import urlopen
from collections import OrderedDict
url = "https://sheets.googleapis.com/$discovery/rest?version=v4"

r = urlopen(url).read()
import json
j=json.loads(r.decode(), object_hook=OrderedDict)

def dump_l(l, indent=0):
    for item in l:
        if isinstance(item, list):
            dump_l(item, indent+1)
        elif isinstance(item, dict):
            dump_d(item, indent+1)
        else:
            print(indent*" ", item, sep="")
        
def dump_d(d, indent=0):
    for k, v in d.items():
        if isinstance(v, dict):
            print(indent*" ", k, ":", sep="")
            dump_d(v, indent+1)
        elif isinstance(v, list):
            dump_l(v, indent+1)
        else:
            assert isinstance(v, (int, str)), type(v)
            print(indent*" ", k, ': ', v, sep="")
            
def dump_resources(d, indent=0):
    for k, v in d.items():
        if k == 'resources':
            for k2, v2 in v.items():
                print(indent*" ", k2, ":", sep="")
                dump_resources(v2, indent+1)

In [2]:
dump_resources(j)

spreadsheets:
 values:
 sheets:


In [3]:
%cd ../pyprojects/goffice3

C:\Users\Administrator\Documents\pyprojects\goffice3


In [4]:
# %load sheets/api.py
"""

Created by: Nathan Starkweather
Created on: 06/10/2016
Created in: PyCharm Community Edition


"""
__author__ = 'Nathan Starkweather'

import logging

logger = logging.getLogger(__name__)
_h = logging.StreamHandler()
_f = logging.Formatter("%(created)s %(name)s %(levelname)s (%(lineno)s): %(message)s")
_h.setFormatter(_f)
logger.addHandler(_h)
logger.propagate = False
logger.setLevel(logging.DEBUG)
del _h, _f

from goffice3 import GSession
from goffice3 import GObject
import json


SPREADSHEETS_URL_BASE = "https://sheets.googleapis.com/v4/spreadsheets"

url_join = "/".join
ajoin = "&".join
eqjoin = "=".join
def _pc(c): return json.loads(c.decode('ascii'))


def make_query(options):
    kvs = [eqjoin((k, v)) for k, v in options if v is not None]
    return "?"+ajoin(kvs)


class Dimension(GObject):
    """ Dimension enum """
    DIMENSION_UNSPECIFIED = "DIMENSION_UNSPECIFIED"
    ROWS = "ROWS"
    COLUMNS = "COLUMNS"


class ValueRenderOption(GObject):
    """ ValueRenderOption enum """
    FORMATTED_VALUE = "FORMATTED_VALUE"
    UNFORMATTED_VALUE = "UNFORMATTED_VALUE"
    FORMULA = "FORMULA"


class DateTimeRenderOption(GObject):
    SERIAL_NUMBER = "SERIAL_NUMBER"
    FORMATTED_STRING = "FORMATTED_STRING"


# brevity
D = Dimension
V = ValueRenderOption
DT = DateTimeRenderOption


class SheetsAPI(GObject):
    """ This API layer implements the raw google sheets
    v4 api via 1-1 translation from python to the REST
    api endpoint(s).

    This api is defined as a hierarchy of collections in
    the same way as the actual google api, to make
    implementation directly reflect the google api structure.

    All Api subobjects (spreadsheets, values...) retain reference
    to the top level api object as a just-in-case implemented in early
    programming.
    """

    def __init__(self, session):
        self.session = session
        self.spreadsheets = SpreadsheetsCollectionAPI(session)

    @classmethod
    def from_creds(cls, creds):
        return cls(GSession(creds))


class SpreadsheetsCollectionAPI(GObject):
    def __init__(self, session):
        self.session = session
        self.sheets = SheetsCollectionAPI(session)
        self.values = ValuesColletionAPI(session)

    def raw_batchUpdate(self, ssid, updates):
        raise NotImplementedError

    def raw_create(self):
        raise NotImplementedError

    def raw_get(self):
        raise NotImplementedError


class ValuesColletionAPI(GObject):
    def __init__(self, session):
        self.session = session

    def raw_batchGet(self, ssid):
        raise NotImplementedError

    def raw_batchUpdate(self, ssid):
        raise NotImplementedError

    def raw_get(self, ssid, ss_range, majorDimension=None, valueRenderOption=None,
            dateTimeRenderOption=None):
        query_options = (
            ('majorDimension', majorDimension),
            ('valueRenderOption', valueRenderOption),
            ('dateTimeRenderOption', dateTimeRenderOption)
        )
        query = make_query(query_options)

        url = url_join((SPREADSHEETS_URL_BASE, str(ssid), "values", ss_range, query))
        r, c = self.session.get(url)
        return _pc(c)

    def raw_update(self, ssid, ss_range):
        raise NotImplementedError


class SheetsCollectionAPI(GObject):
    def __init__(self, session):
        self.session = session

    def raw_copyTo(self):
        raise NotImplementedError


FOOFOOFOOFOOFOOFOOFOOFOOFOOFOO


In [5]:
def parse_content(c):
    return json.loads(c.decode('ascii'))

import os
from goffice3 import load_creds, GSession
import json
os.chdir(os.path.expanduser("~/Documents/pyprojects/google_auths/.auth"))
creds = load_creds('_exp_auth_key.json')
s = GSession(creds)
api = SheetsAPI(s)

dv3u = "https://www.googleapis.com/drive/v3"
dv3fileslist = dv3u+"/files"
r, c = s.get(dv3fileslist)
j=json.loads(c.decode())

ssid = j['files'][0]['id']
j['files']

c = api.spreadsheets.values.raw_get(ssid, "A2")
c2 = api.spreadsheets.values.raw_get(ssid, "A5")

1465774571.110973 goffice3.baseobject DEBUG (47): New instance of <GSession>.
1465774571.112973 goffice3.baseobject DEBUG (47): New instance of <SheetsAPI>.
1465774571.114973 goffice3.baseobject DEBUG (47): New instance of <SpreadsheetsCollectionAPI>.
1465774571.115973 goffice3.baseobject DEBUG (47): New instance of <SheetsCollectionAPI>.
1465774571.117974 goffice3.baseobject DEBUG (47): New instance of <ValuesColletionAPI>.
1465774571.119974 goffice3.session DEBUG (71): Refreshing authentication key
1465774574.536194 goffice3.session DEBUG (76): GET: https://www.googleapis.com/drive/v3/files
1465774574.811212 goffice3.session DEBUG (76): GET: https://sheets.googleapis.com/v4/spreadsheets/10x9ziXbfVm5ZcchAZjW1Zi7L0c0tDJfPhIvsnryvQZY/values/A2/?
1465774575.24924 goffice3.session DEBUG (76): GET: https://sheets.googleapis.com/v4/spreadsheets/10x9ziXbfVm5ZcchAZjW1Zi7L0c0tDJfPhIvsnryvQZY/values/A5/?


In [6]:
j['files'][0]

{'id': '10x9ziXbfVm5ZcchAZjW1Zi7L0c0tDJfPhIvsnryvQZY',
 'kind': 'drive#file',
 'mimeType': 'application/vnd.google-apps.spreadsheet',
 'name': 'Slayer Tasks'}

In [7]:
DRIVE_URL_V3 = "https://www.googleapis.com/drive/v3"
DRIVE_URL_V2 = "https://www.googleapis.com/drive/v2"

class DriveAPI(GObject):
    def __init__(self, session):
        self.session = session
        self.Files = FilesAPI(self, session)
        
class FilesAPI(GObject):
    def __init__(self, drive, session):
        self.drive = drive
        self.session = session
        
    def list(self, maxResults=None, orderBy=None, pageToken=None, q=None, spaces=None, corpus=None):
        """ https://developers.google.com/drive/v2/reference/files/list """
        ops = (
            ('maxResults', maxResults),
            ("orderBy", orderBy),
            ("pageToken", pageToken),
            ("q", q),
            ("spaces", spaces),
            ("spaces", corpus)
        )
        query = make_query(ops)
        url = url_join((DRIVE_URL_V3, 'files', query))
        r, c = self.session.get(url)
        return _pc(c)

In [8]:
class DataObject(GObject):
    def __init__(self, api, js=None):
        """ 
        :type js: json object with file attributes 
        """
        if not isinstance(js, (type(None), dict)):
            print(js)
        self._js_ob = js or {}
        self._api = api
        
    def _js_get(self, key):
        try:
            return self._js_ob[key]
        except KeyError:
            raise AttributeError(key) from None
                    
    
class DriveFiles(DataObject):
    def __init__(self, api, ob):
        super().__init__(api)
        self._dfs = [DriveFile(api, f) for f in ob]
        self._files = {df.name: df for df in self._dfs}
        
    def __getitem__(self, key):
        if isinstance(key, int):
            return self._dfs[key]
        elif isinstance(key, str):
            return self._files[key]
        else:
            raise TypeError(key)
            
    def __iter__(self):
        return iter(self._dfs)
    
    def __repr__(self):
        tmpl = "%s(\n %s\n)"
        l = ',\n '.join(repr(df) for df in self._dfs)
        return tmpl % (self.__class__.__name__, l)

        
class DriveFile(DataObject):
    @property
    def id(self):
        return self._js_get('id')
    
    @property
    def kind(self):
        return self._js_get('kind')
    
    @property
    def mimeType(self):
        return self._js_get('mimeType')
    
    @property
    def name(self):
        return self._js_get('name')
    
    def __repr__(self):
        return "%s(\"%s\")" % (self.__class__.__name__, self.name)
    
    
class Drive(GObject):
    """ Drive object is the top-level object, so it gets initialized a bit differently """
    
    def __init__(self, auth_key=None):
        if auth_key is not None:
            api = DriveAPI(GSession(auth_key))
        else:
            api = None
        self.api = api

    @classmethod
    def from_session(cls, sess):
        api = DriveAPI(sess)
        self = cls()
        self.api = api
        return self
    
    @classmethod
    def from_api(cls, api):
        self = cls()
        self.api = api
        return self

    @property
    def Files(self):
        files = self.api.Files.list()
        return DriveFiles(self.api, files['files'])
        

In [9]:
d = Drive.from_session(s)
d.Files[0]._js_ob

1465774575.943285 goffice3.baseobject DEBUG (47): New instance of <DriveAPI>.
1465774575.944285 goffice3.baseobject DEBUG (47): New instance of <FilesAPI>.
1465774575.946285 goffice3.baseobject DEBUG (47): New instance of <Drive>.
1465774575.948285 goffice3.session DEBUG (76): GET: https://www.googleapis.com/drive/v3/files/?
1465774576.128297 goffice3.baseobject DEBUG (47): New instance of <DriveFiles>.
1465774576.130297 goffice3.baseobject DEBUG (47): New instance of <DriveFile>.
1465774576.131297 goffice3.baseobject DEBUG (47): New instance of <DriveFile>.
1465774576.133297 goffice3.baseobject DEBUG (32): <weakref at 0x0000000004D1FEA8; to 'DriveFiles' at 0x0000000005BB0400> object destroyed.
1465774576.135297 goffice3.baseobject DEBUG (32): <weakref at 0x0000000004D15AE8; to 'DriveFile' at 0x0000000005BB01D0> object destroyed.
1465774576.136297 goffice3.baseobject DEBUG (32): <weakref at 0x000000000672A408; to 'DriveFile' at 0x0000000005BB0198> object destroyed.


{'id': '10x9ziXbfVm5ZcchAZjW1Zi7L0c0tDJfPhIvsnryvQZY',
 'kind': 'drive#file',
 'mimeType': 'application/vnd.google-apps.spreadsheet',
 'name': 'Slayer Tasks'}

In [10]:
def reload():
    for m in list(sys.modules):
        if 'goffice3' in m:
            del sys.modules[m]

In [11]:
from goffice3.session import *